# Generate Train txt file for thunlp/TensorFlow-NRE from wikipedia-biography dataset

In [1]:
# Checking Python Version 3+ 
import sys
print(sys.version)

3.6.1 |Anaconda custom (64-bit)| (default, May 11 2017, 13:09:58) 
[GCC 4.4.7 20120313 (Red Hat 4.4.7-1)]


## Link files

train.title -1-1-> train.box -1-1-> train.nb -[number]-1-> train.sent


## Resule Schema:
title, non_na_box, accumulated_sent_context

## Sample Training Data from thunlp/TensorFlow-NRE:
#### Format: (fb_mid_e1, fb_mid_e2, e1_name, e2_name, relation, sentence)   

### Sample:    

fb_mid_e1 - m.0ccvx   
fb_mid_e2 - m.05gf08   
e1_name   - queens   
e2_name   - belle_harbor   
relation  - /location/location/contains   
sentence  - sen. charles e. schumer called on federal safety officials yesterday to reopen their investigation into the fatal crash of a passenger jet in belle_harbor , queens , because equipment failure , not pilot error , might have been the cause . ###END###   


In [1]:
# wikipedia biography training files
# substitute test and validate files
data_type = 'train' # test, valid
train_title_file = data_type + '/' + data_type + ".title"
train_nb_file    = data_type + '/' + data_type + ".nb"
train_sent_file  = data_type + '/' + data_type + ".sent"
train_box_file   = data_type + '/' + data_type + ".box"

In [2]:
# add line indexer for the sent file
nbs_dict = {}
with open(train_sent_file) as sent:
    for line, content in enumerate(sent):
        nbs_dict[line] = content
    
nbs_dict.get(0)

'walter extra is a german award-winning aerobatic pilot , chief aircraft designer and founder of extra flugzeugbau -lrb- extra aircraft construction -rrb- , a manufacturer of aerobatic aircraft .\n'

In [3]:
# remove words from a context string, i.e., -lrb-
useless_words_to_remove = ['-lrb-', '-rrb-', '.\n', '']

def cleanUpSentence(input_sent):
    keywords_to_remove = useless_words_to_remove
    querywords = input_sent.split()
    resultwords  = [word for word in querywords if word.lower() not in keywords_to_remove]
    result = ' '.join(resultwords)
    return result

In [4]:
send_index = 0

sent_dict = {}

with open(train_nb_file) as nbs:
    accumulated_lines = 0
    for nb in nbs:
        current_lines_to_read = int(nb)
        
        current_accumulated_sent_context = ''
        
        for i in range(current_lines_to_read):
            line_to_read = accumulated_lines + i
            current_accumulated_sent_context += cleanUpSentence(nbs_dict.get(line_to_read))
        
        accumulated_lines += current_lines_to_read

        sent_dict[send_index] = current_accumulated_sent_context
        send_index += 1      

In [5]:
sent_dict[1]

'aaron hohlbein born august 16 , 1985 in middleton , wisconsin is an american soccer player who is currently without a club .'

## Get top 100 most shown relationships

In [6]:
import re
from collections import defaultdict

In [7]:
relations_stat_dict = defaultdict(int)

with open(train_box_file) as boxes:
    for one_entry in boxes:
        
        all_target_attributes = re.split(r'\t+', one_entry)
        
        filtered_attrs = [attr for attr in all_target_attributes if '<none>' not in attr]
        
        for attr in filtered_attrs:
            if '_1:' in attr:
                attr_label = attr.split(':')[0][:-2]
                relations_stat_dict[attr_label] += 1

In [9]:
sumVal = 0
for i in relations_stat_dict:
    value = relations_stat_dict[i]
    

name
birth_date
nationality
occupation
article_title
fullname
birth_place
height
position
youthyears
youthclubs
years
clubs
caps
goals
pcupdate
image
death_date
death_place
death_cause
resting_place
other_names
known_for
education
alma_mater
party
birthname
originalteam
debutdate
debutteam
debutopponent
debutstadium
statsend
currentclub
clubnumber
nationalyears
nationalteam
nationalcaps
nationalgoals
ntupdate
imagesize
office
term_start
term_end
predecessor
successor
spouse
caption
batting
bowling
columns
column
matches
runs
s/s
deliveries
wickets
fivefor
tenfor
catches/stumpings
international
country
testdebutdate
testdebutyear
lasttestdate
lasttestyear
source
date
year
jr/sr
state
alongside
leader
deputy
state_senate
district
religion
website
manageryears
managerclubs
image_size
awards
parents
children
bgcolour
field
training
background
instrument
years_active
birth_name
bats
throws
debutyear
finaldate
finalyear
finalteam
statlabel
statvalue
disciplines
club
skis
olympicteams
olympic

members_number
legislature_status
opposition_party
legislature_term
budget
previous
license_place
fansite
honorary_suffix
compensation
article
portion
replaceability
studio
viceroy
current
|influences
aflrookieyear
faith
years_of_service
beliefs
yearend|appearances
px|monuments
inseam
former_name
woman
played
alma_mata
institute
fieldgoalsba
|children
yearfend
academic
dharma_name
reincarnation_of
town
political_views
ideology
know_for
native
pages
oclc
past
building_name
geo
religious_affiliation
functional_status
architecture
architecture_type
architecture_style
year_completed
specifications
materials
grandson
rl_currentclub
nobel
studies
image_skyline
map_caption
leader_name
area_magnitude
area_total_km
area_land_km
area_water_km
population_metro
population_total
postal_code_type
postal_code
area_code
latm
latns
longm
longew
cortÉs|office
nikname
l
replacing
horsename
cause_death
military_awards
uniform
venue
torse
live_members
height_present
currentlub
research_interests
grandslamd

In [9]:
top_100_most_shown_relations = {} # relation_label, number_of_times_it_shown
sortedValues = sorted(relations_stat_dict.values(), reverse=True)
max_value = sortedValues[0]
onehundredth_value = sortedValues[99]
for attr_label in relations_stat_dict:
    attr_rep = relations_stat_dict[attr_label]
    if attr_rep >= onehundredth_value:
        top_100_most_shown_relations[attr_label] = attr_rep

In [10]:
print(top_100_most_shown_relations)

{'name': 66734, 'title': 2379, 'predecessor': 6966, 'successor': 6266, 'birth_place': 57282, 'death_date': 21656, 'nationality': 13886, 'religion': 3968, 'residence': 4581, 'article_title': 72831, 'fullname': 13223, 'height': 13002, 'birth_date': 63212, 'currentclub': 6813, 'clubnumber': 4161, 'position': 19150, 'years': 12920, 'clubs': 11739, 'caps': 10374, 'goals': 10775, 'pcupdate': 4466, 'updated': 1845, 'district': 1614, 'term_start': 7742, 'term_end': 6418, 'party': 6705, 'alma_mater': 6891, 'spouse': 8928, 'image': 26790, 'caption': 15578, 'occupation': 18943, 'years_active': 7638, 'image_size': 6682, 'team': 2529, 'number': 2118, 'bats': 2007, 'throws': 2012, 'debutdate': 2070, 'debutyear': 3232, 'debutteam': 2855, 'statlabel': 3041, 'statvalue': 3040, 'background': 5886, 'origin': 3587, 'genre': 6299, 'label': 3775, 'associated_acts': 3161, 'college': 3101, 'height_ft': 2969, 'height_in': 2967, 'imagesize': 2817, 'death_place': 16746, 'birth_name': 9367, 'weight_lb': 2002, 'ca

## Generate the relation2id.txt file

In [11]:
#### Remove the following relation labels for speeding up:
del top_100_most_shown_relations['clubs']
del top_100_most_shown_relations['years']
del top_100_most_shown_relations['image']
del top_100_most_shown_relations['name']

In [12]:
if data_type == 'train':
    relation2idFile = open(data_type+'/relation2id.generate.txt', "w") 
    relation2idFile.write('NA 0\n')
    relationid = 1
    for relation in top_100_most_shown_relations:
        relation2idFile.write(relation+' '+str(relationid)+'\n')
        relationid += 1
    relation2idFile.close()

## Get the Concatenated Relation Labels:
Current, the relation labels are splitted in the wikipedia biography dataset.  
In order to adapt to NRE code, we have to concatenate all splitted labels into one single string with underscore in between.  

In [13]:
real_relation_label_and_value_list = []

counter = 0

with open(train_box_file) as boxes:
    for one_entry in boxes:
        
        all_target_attributes = re.split(r'\t+', one_entry)
        filtered_attrs = [attr for attr in all_target_attributes if '<none>' not in attr]
        
        current_box_dict = {}
        for oneLabel in filtered_attrs:
            labelStringOnly = oneLabel.split(':')[0].split('_')[0]
            if labelStringOnly in top_100_most_shown_relations:
                currentValue = cleanUpSentence(oneLabel.split(':')[1])
                if labelStringOnly in current_box_dict:
                    current_box_dict[labelStringOnly] = current_box_dict[labelStringOnly] + '_' + currentValue
                else:
                    current_box_dict[labelStringOnly] = currentValue
               
        real_relation_label_and_value_list.append(current_box_dict)

In [14]:
print(len(real_relation_label_and_value_list))
print(len(real_relation_label_and_value_list) == len(sent_dict))

72831
True


## Generate the training data txt file:

#### Read the titles into a list, in original order:

In [15]:
title_list = []
with open(train_title_file) as titles:
    for title in titles:
        title_list.append(title[:-1])

#### Verify that the three lists (title_list, sent_dict, and real_relation_label_and_value_list) start with index 0:

In [16]:
title_list[0]

'pope michael iii of alexandria'

In [17]:
sent_dict[0]

'pope michael iii of alexandria also known as khail iii was the coptic pope of alexandria and patriarch of the see of st. mark 880 -- 907 .in 882 , the governor of egypt , ahmad ibn tulun , forced khail to pay heavy contributions , forcing him to sell a church and some attached properties to the local jewish community .this building was at one time believed to have later become the site of the cairo geniza .'

In [18]:
for i in real_relation_label_and_value_list[0]:
    print(real_relation_label_and_value_list[0][i])

56th_pope_of_alexandria_&_patriarch_of_the_see_of_st._mark
shenouda_i
gabriel_i
egyptian
coptic_orthodox_christian
saint_mark_'s_church


#### Join three list by the biography title and write to text file:
This is going to generate a 1+ GB large file

In [19]:
resultFile = open(data_type+'/'+data_type+".generate.txt", "w") 

index = 0

for relationAndValueEntry in real_relation_label_and_value_list:
    for relationName in relationAndValueEntry:
        resultFile.write(title_list[index])
        resultFile.write('\t')
        resultFile.write(relationAndValueEntry[relationName])
        resultFile.write('\t')
        resultFile.write(relationName)
        resultFile.write('\t')
        resultFile.write(sent_dict[index])
        resultFile.write(' ###END###\n')
    index += 1

resultFile.close()

## Generate the relation2id.txt file